In [ ]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

wf_md5 = "md5"
wf_fastqc = "fastqc-0-11-4-1"

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
run_md_qc = raw_input("Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)")

# status for completion
# there are two flavors of complete signals, before it was output_file_transfer_finished, not it is complete.
# old completed wf runs have former one.

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

################
##ADD TO WORKFLOW
# wfr_time = datetime.strptime(wfr_data['date_created'],'%Y-%m-%dT%H:%M:%S.%f+00:00')
# run_hours = int((datetime.now()-wfr_time).total_seconds()/3600)
################

def get_files(exp_set_id):
    files = []
    exps = ff_utils.get_metadata(exp_set_id, connection=ff)['experiments_in_set']
    for an_exp in exps:
        exp_resp = ff_utils.get_metadata(an_exp, connection=ff)['files']
        files.extend(exp_resp)
    return files

def summarize_file(file_resp):
    md5 = False
    qc = False
    file_id = file_resp['accession']
    sequencer = file_resp.get('instrument')
    relations = file_resp.get('related_files')
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    first_alias = file_resp.get('aliases',[None])[0]
    pair_no = file_resp.get('paired_end')
    # is md5 fine
    if file_resp.get('content_md5sum'):
        md5 = True
    # is there md5sum for gzip
    if not file_resp.get('md5sum'):
        print file_id,"does not have the md5sum calculated during upload"
        
    # is there a qc?
    if file_resp.get('quality_metric'):
        qc = True
    # Check workflows for qc fastqc workflow partA
    md5_status = 'did_not_run'
    fastqc_status = 'did_not_run'
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for a_wfr in workflows:
            wfr_resp = ff_utils.get_metadata(a_wfr['uuid'], connection=ff)
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_fastqc):
                fastqc_status = wfr_resp.get('run_status') 
            
                
    # Check for md5 and fastqc, and if not complete, run or report it. 
    # if exclude miseq is on, do this only if sequencer is not miseq
    if not md5 or status in ["uploading", "upload failed"] or md5_status != 'complete':
        # if not, shall we run it?
        if run_md_qc in ['md5', 'all']:
            print 'md5 running for', file_resp['accession']
            code_md5= "invoke run_md5 " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(3)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_resp['accession']
    # check fastqc if md5 is fine
    else:
        if not qc or fastqc_status != 'complete':
            # if not, shall we run it?
            if run_md_qc in ['qc', 'all']:
                print 'fastqc running for', file_resp['accession']
                code_qc= "invoke run_fastqc " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                run(code_qc)
                print ''    
                time.sleep(3)
            # user does not want it to be run, so just report
            else:
                print 'fastqc run missing for', file_resp['accession'], fastqc_status
                print 
    return



all_files = ff_utils.get_metadata('files-fastq', connection=ff)['@graph']
print len(all_files)

printn = 0
counter = 0
for a_file in all_files:  
    counter += 1
    # check for deleted or weird cases
    try:
        if a_file['status'] == 'deleted':
            #print "Deleted File", a_file
            continue
    except:
        print a_file
        break
        
    if counter-printn > 100:
        print counter
        printn = counter

    file_resp = ff_utils.get_metadata(a_file['uuid'], connection=ff, frame='embedded')
    # check if file is in s3
    head_info = tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket)
    if not head_info:
        print file_resp['accession'], "does not have a file in S3"
        continue

    file_info = summarize_file(file_resp)
    
 


Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)qc
3876
fastqc running for 4DNFI89NERUM
about to start run fastqc-0-11-4-1_4DNFI89NERUM.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 16, 20, 423000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '8bea2064-1bd9-11e8-94b2-1f2b103a9b46', 'HTTPHeaders': {'x-amzn-requestid': '8bea2064-1bd9-11e8-94b2-1f2b103a9b46', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFI89NERUM.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFI89NERUM.fastq.gz

fastqc running for 4DNFIUA6IWTC
about to start run fastqc-0-11-4-1_4DNFIUA6IWTC.fastq.gz
response from aws was: 
 {u'startDate': datetime.datet


fastqc running for 4DNFILX1MDCI
about to start run fastqc-0-11-4-1_4DNFILX1MDCI.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 17, 26, 484000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'b349ee4f-1bd9-11e8-bf87-e396191f33a8', 'HTTPHeaders': {'x-amzn-requestid': 'b349ee4f-1bd9-11e8-bf87-e396191f33a8', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFILX1MDCI.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFILX1MDCI.fastq.gz

fastqc running for 4DNFIT95DVA3
about to start run fastqc-0-11-4-1_4DNFIT95DVA3.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 17, 32, 793000, tzinfo=tzlocal()), 'ResponseMetada


fastqc running for 4DNFIG6HGIMH
about to start run fastqc-0-11-4-1_4DNFIG6HGIMH.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 18, 37, 336000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'dd860088-1bd9-11e8-84c7-cd28dcc0c68b', 'HTTPHeaders': {'x-amzn-requestid': 'dd860088-1bd9-11e8-84c7-cd28dcc0c68b', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIG6HGIMH.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIG6HGIMH.fastq.gz

fastqc running for 4DNFIGBBECDZ
about to start run fastqc-0-11-4-1_4DNFIGBBECDZ.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 18, 43, 872000, tzinfo=tzlocal()), 'ResponseMetada


fastqc running for 4DNFIJMA8Z8T
about to start run fastqc-0-11-4-1_4DNFIJMA8Z8T.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 19, 47, 8000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '070b9483-1bda-11e8-9556-71885d00b82c', 'HTTPHeaders': {'x-amzn-requestid': '070b9483-1bda-11e8-9556-71885d00b82c', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIJMA8Z8T.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIJMA8Z8T.fastq.gz

fastqc running for 4DNFI13FR88Q
about to start run fastqc-0-11-4-1_4DNFI13FR88Q.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 19, 53, 84000, tzinfo=tzlocal()), 'ResponseMetadata'


fastqc running for 4DNFI65KUPD5
about to start run fastqc-0-11-4-1_4DNFI65KUPD5.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 21, 9, 755000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '385d5044-1bda-11e8-93cc-91ba304435d7', 'HTTPHeaders': {'x-amzn-requestid': '385d5044-1bda-11e8-93cc-91ba304435d7', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFI65KUPD5.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFI65KUPD5.fastq.gz

fastqc running for 4DNFIKQTO7TX
about to start run fastqc-0-11-4-1_4DNFIKQTO7TX.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 21, 19, 369000, tzinfo=tzlocal()), 'ResponseMetadat

...


fastqc running for 4DNFICH8UG6P
about to start run fastqc-0-11-4-1_4DNFICH8UG6P.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 40, 38, 738000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'f1241a2d-1bdc-11e8-bb89-fb1653298ed2', 'HTTPHeaders': {'x-amzn-requestid': 'f1241a2d-1bdc-11e8-bb89-fb1653298ed2', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFICH8UG6P.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFICH8UG6P.fastq.gz

fastqc running for 4DNFIYSS7ZWV
about to start run fastqc-0-11-4-1_4DNFIYSS7ZWV.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 40, 54, 73000, tzinfo=tzlocal()), 'ResponseMetadat


fastqc running for 4DNFIAVDV82A
about to start run fastqc-0-11-4-1_4DNFIAVDV82A.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 42, 49, 573000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '3f1cbbb9-1bdd-11e8-a936-73c6ee55a392', 'HTTPHeaders': {'x-amzn-requestid': '3f1cbbb9-1bdd-11e8-a936-73c6ee55a392', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIAVDV82A.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIAVDV82A.fastq.gz

fastqc running for 4DNFIV6W4J3B
about to start run fastqc-0-11-4-1_4DNFIV6W4J3B.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 42, 59, 292000, tzinfo=tzlocal()), 'ResponseMetada


fastqc running for 4DNFIC2O5ES8
about to start run fastqc-0-11-4-1_4DNFIC2O5ES8.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 44, 38, 137000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '7fd751d8-1bdd-11e8-88da-6df3ea2e2e6d', 'HTTPHeaders': {'x-amzn-requestid': '7fd751d8-1bdd-11e8-88da-6df3ea2e2e6d', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIC2O5ES8.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIC2O5ES8.fastq.gz

fastqc running for 4DNFIJA25O49
about to start run fastqc-0-11-4-1_4DNFIJA25O49.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 44, 47, 487000, tzinfo=tzlocal()), 'ResponseMetada


fastqc running for 4DNFIXA1TD2E
about to start run fastqc-0-11-4-1_4DNFIXA1TD2E.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 46, 19, 870000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'bc792b4a-1bdd-11e8-87d3-55e944443671', 'HTTPHeaders': {'x-amzn-requestid': 'bc792b4a-1bdd-11e8-87d3-55e944443671', 'content-length': '145', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIXA1TD2E.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIXA1TD2E.fastq.gz

fastqc running for 4DNFIISX7GKO
about to start run fastqc-0-11-4-1_4DNFIISX7GKO.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 46, 29, 654000, tzinfo=tzlocal()), 'ResponseMetada


fastqc running for 4DNFIXMHZCJI
about to start run fastqc-0-11-4-1_4DNFIXMHZCJI.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 48, 15, 583000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '016e374e-1bde-11e8-9284-f18ba3d577da', 'HTTPHeaders': {'x-amzn-requestid': '016e374e-1bde-11e8-9284-f18ba3d577da', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIXMHZCJI.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIXMHZCJI.fastq.gz

202
fastqc running for 4DNFISF7LTKK
about to start run fastqc-0-11-4-1_4DNFISF7LTKK.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 2, 27, 11, 48, 24, 971000, tzinfo=tzlocal()), 'ResponseMe